In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import metrics

from operator import itemgetter

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.neighbors import NearestNeighbors

from sklearn.linear_model import LogisticRegression

In [2]:
DF = pd.read_csv("citeulike-a/raw-data.csv",encoding='ISO-8859-1')#'ISO-8859-1')

In [116]:
DF.head()

NameError: name 'DF' is not defined

In [4]:
DF.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...
16978,16979,software that goes with the flow in systems bi...,8331304.0,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...
16979,16980,a bacterium that can grow by using arsenic ins...,8349297.0,A Bacterium That Can Grow by Using Arsenic Ins...,Life is mostly composed of the elements carbon...


In [5]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16980 entries, 0 to 16979
Data columns (total 5 columns):
doc.id          16980 non-null int64
title           16980 non-null object
citeulike.id    16980 non-null float64
raw.title       16980 non-null object
raw.abstract    16980 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 663.4+ KB


In [6]:
citations = pd.read_csv("citeulike-a/citations.dat",delimiter='\n')

In [7]:
citations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16981 entries, 0 to 16980
Data columns (total 1 columns):
citations    16981 non-null object
dtypes: object(1)
memory usage: 132.7+ KB


In [8]:
citations.tail()

,citations
16976,"16,420,6357,6371,9466,12007,12096,12660,13307,..."
16977,"8,7991,15184,15944,16272,16304,16451,16619,16722"
16978,"14,418,10406,10558,11796,14709,15093,15248,152..."
16979,0
16980,0


In [9]:
citations2=citations[1:len(citations)-1].copy()

In [10]:
#citations2.index.rename('doc.id',inplace=True)

In [11]:
citations2.reset_index(inplace=True)

In [12]:
citations2.tail()

,index,citations
16974,16975,"12,1631,10468,10587,11949,12665,13198,14527,14..."
16975,16976,"16,420,6357,6371,9466,12007,12096,12660,13307,..."
16976,16977,"8,7991,15184,15944,16272,16304,16451,16619,16722"
16977,16978,"14,418,10406,10558,11796,14709,15093,15248,152..."
16978,16979,0


In [13]:
DF2=pd.concat([DF,citations2],axis=1)

In [14]:
type(DF2['citations'][0])

str

In [15]:
N = 10

In [16]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,"3,2,485,3284"
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,"16,42,43,60,113,116,161,252,1543,1782,1947,414..."
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,"85,0,4,5,10,11,15,23,27,28,48,52,79,106,368,46..."
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,0
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,"23,2,28,488,918,1200,1351,1861,1937,2289,3079,..."


In [99]:
DF2.tail()

,doc.id,raw.title,raw.abstract
16974,16975,Analysing biological pathways in genome-wide a...,Genome-wide association (GWA) studies have ty...
16975,16976,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...
16976,16977,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...
16977,16978,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...
16978,16979,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...


In [17]:
DF2.dropna(inplace=True,axis=0)

In [18]:
DF2['citations']=DF2['citations'].apply(lambda x: x.split(','))

In [19]:
del DF
del citations
del citations2

In [20]:
IdFreq={}

In [21]:
def docIdFreq (c_list):
    for element in c_list:
        if element in IdFreq:
            IdFreq[element] = IdFreq[element]+1
        else:
            IdFreq[element] = 1

In [22]:
DF2['citations'].apply(lambda x: docIdFreq(x));

In [23]:
most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)

In [24]:
del most_popular[0]

In [25]:
most_popular[0:N]

[('1', 1732),
 ('2', 1585),
 ('3', 1238),
 ('4', 1076),
 ('5', 937),
 ('6', 747),
 ('7', 618),
 ('8', 521),
 ('9', 493),
 ('10', 402)]

In [26]:
DF2.drop('index',axis=1,inplace=True)

In [27]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,"[3, 2, 485, 3284]"
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,"[16, 42, 43, 60, 113, 116, 161, 252, 1543, 178..."
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...","[85, 0, 4, 5, 10, 11, 15, 23, 27, 28, 48, 52, ..."
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,[0]
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,"[23, 2, 28, 488, 918, 1200, 1351, 1861, 1937, ..."


In [28]:
DF2[DF2['doc.id']==1]['raw.title'].values[0]

'The metabolic world of Escherichia coli is not small'

# Most popular article 

'The metabolic world of Escherichia coli is not small'

In [29]:
UR = pd.read_csv("citeulike-a/users.dat",delimiter='\n',names=['liked_articles'],header=None)

In [30]:
UR.head()

,liked_articles
0,70 495 1631 2317 2526 2846 2931 3171 3297 3332...
1,38 493 942 1519 1843 1844 1896 2819 3391 5031 ...
2,20 517 791 800 1329 1767 1984 2126 3009 4458 4...
3,12 706 709 721 755 756 776 892 895 2588 2991 4...
4,12 761 2678 4034 4035 5439 5494 5758 7798 9046...


In [31]:
UR['liked_articles']=UR['liked_articles'].apply(lambda x: x.split(' '))

In [32]:
UR.head()

,liked_articles
0,"[70, 495, 1631, 2317, 2526, 2846, 2931, 3171, ..."
1,"[38, 493, 942, 1519, 1843, 1844, 1896, 2819, 3..."
2,"[20, 517, 791, 800, 1329, 1767, 1984, 2126, 30..."
3,"[12, 706, 709, 721, 755, 756, 776, 892, 895, 2..."
4,"[12, 761, 2678, 4034, 4035, 5439, 5494, 5758, ..."


In [33]:
IdFreq={}

In [34]:
UR['liked_articles'].apply(lambda x: docIdFreq(x));

In [35]:
most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)

In [36]:
del most_popular[0]

In [37]:
most_popular = most_popular[0:N]

In [38]:
mp_docId = [ element[0] for element in most_popular]

In [39]:
DF2[DF2['doc.id'].isin(mp_docId)]['raw.title'].values

array(['Organization, development and function of complex brain networks',
       'Motifs in brain networks.',
       'Diffusion on Complex Networks : A way to probe their large scale topological structures',
       'Topological Generalizations of network motifs',
       "Collective dynamics of 'small-world' networks.",
       'Network motifs: simple building blocks of complex networks.',
       'Authoritative sources in a hyperlinked environment',
       'Finding and evaluating community structure in networks',
       'Subgraph centrality in complex networks.',
       'Finding structure with randomness: Stochastic algorithms for constructing approximate matrix decompositions'], dtype=object)

# Random

In [40]:
d_id=np.random.randint(low=0, high=len(DF2)-1,size=N)

In [41]:
DF2[DF2['doc.id'].isin(d_id)]['raw.title'].values

array(['Automatic Meaning Discovery Using Google',
       'A genomic regulatory network for development.',
       'The Presentation of Self in Everyday Life',
       'Greedy Function Approximation: A gradient boosting machine',
       'Vehicles, Experiments in Synthetic Psychology',
       'Coevolution of genomic intron number and splice sites',
       'Human Genome Ultraconserved Elements Are Ultraselected',
       'Population genomics of domestic and wild yeasts',
       'reCAPTCHA: Human-Based Character Recognition via Web Security Measures',
       'phyloXML: XML for evolutionary biology and comparative genomics'], dtype=object)

# Content-Based

In [42]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [43]:
tfidf_matrix = tfidf_vectorizer.fit_transform(DF2['raw.abstract'])
nbrs_tfidf        = NearestNeighbors(n_neighbors=N+1,metric='cosine').fit(tfidf_matrix)

In [44]:
def get_closest_neighbors_tfidf(title):
    row                = DF2[DF2['raw.title']==title].index[0]
    distances, indices = nbrs_tfidf.kneighbors(tfidf_matrix.getrow(row))
    titles_similar      = pd.Series(indices.flatten()).map(DF2.reset_index()['raw.title'])
    result             = pd.DataFrame({'distance':distances.flatten(), 'raw.title':titles_similar})
    return result

In [45]:
get_closest_neighbors_tfidf('What is the goal of sensory coding')['raw.title'].values[1:N+1]

array([ 'Emergence of simple-cell receptive field properties by learning a sparse code for natural images',
       'Relations between the statistics of natural images and the response properties of cortical cells',
       'Sparse coding with an overcomplete basis set: A strategy employed by V',
       'Sparse coding and decorrelation in primary visual cortex during natural vision',
       'Natural image statistics and efficient coding',
       "The `Independent Components' of Natural Scenes are Edge Filters",
       'Cortical Sensitivity to Visual Features in Natural Scenes',
       'Statistical learning of new visual feature combinations by infants.',
       'Sparse Coding of Sensory Inputs',
       'Adaptive filtering enhances information transmission in visual cortex.'], dtype=object)

# Collaborative FiItering

In [98]:
docId = DF2['doc.id']

In [97]:
docId[len(docId)-1]

16979

In [48]:
del CountVectorizer
del Counter
del IdFreq
del LogisticRegression
del TfidfVectorizer
del tfidf_matrix
del nbrs_tfidf
del tfidf_vectorizer

In [ ]:
DF2.drop(['citeulike.id','title','citations'],axis=1,inplace=True)

In [100]:
M = np.zeros(shape=(len(UR),len(docId)))

for i, row in UR.iterrows():
    for j in range(len(row['liked_articles'])-1):
        M[i,int(row['liked_articles'][j])]=1

In [ ]:
BARRIER();

In [101]:
#del UR
del docId

In [102]:
user_ratings_mean = np.sum(M, axis = 1)

In [103]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(M, k = 500)

In [104]:
sigma = np.diag(sigma)

In [105]:
print(U.shape,Vt.shape)

(5551, 500) (500, 16979)


In [106]:
all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))# + 
                              #user_ratings_mean.reshape(-1, 1))/user_ratings_mean.reshape(-1, 1)

In [107]:
preds_df = pd.DataFrame(all_user_predicted_ratings)

## Recommending for user #:

In [108]:
user_row_number=598

In [109]:
sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

In [110]:
preds_df.head()

,0,1,2,3,4,5,6,7,8,9,...,16969,16970,16971,16972,16973,16974,16975,16976,16977,16978
0,-0.108217,-0.012148,0.207660,0.116254,-0.018404,-0.008025,0.025870,-0.003720,0.013988,0.008246,...,-0.007290,0.002763,-0.004214,0.006010,-0.002886,0.012461,-0.007043,0.0,0.0,-0.004424
1,0.006121,-0.004489,-0.022790,0.110287,-0.032111,0.000457,0.008041,0.015893,0.014139,0.003591,...,-0.005871,-0.006162,0.001106,-0.000193,-0.006080,-0.011805,0.001606,0.0,0.0,-0.001275
2,0.005672,-0.011155,-0.003710,0.007297,0.036504,-0.006043,0.002950,-0.006832,0.003880,0.000958,...,0.000882,-0.000410,-0.005494,0.001048,-0.001947,-0.000304,0.002074,0.0,0.0,-0.000467
3,-0.000598,0.007160,-0.018280,0.011815,0.000643,0.007045,-0.000263,0.007097,0.000032,0.004871,...,-0.001925,-0.000018,-0.000692,-0.000693,-0.001710,-0.000841,0.002519,0.0,0.0,-0.000182
4,0.003032,0.018895,0.009498,0.009376,-0.005467,0.002482,0.006292,0.003004,0.010248,0.001656,...,-0.001217,-0.001085,0.001103,-0.001046,0.001163,-0.002314,0.002145,0.0,0.0,0.000367


In [111]:
recommendations = (DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].
merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'doc.id',
               right_on = 'index'))

In [112]:
recommendations.rename(columns = {user_row_number: 'Predictions'},inplace=True)

In [113]:
recommendations.drop('index',axis=1,inplace=True)

In [114]:
recommendations.sort_values('Predictions', ascending = False,inplace=True)

In [115]:
BARRIER();

NameError: name 'BARRIER' is not defined

In [117]:
# k = 200
recommendations.head(N)

,doc.id,raw.title,raw.abstract,Predictions
6950,6994,An evaluation of GO annotation retrieval for B...,BACKGROUND: The Gene Ontology Annotation (GOA)...,0.135961
16465,16700,Missing heritability and strategies for findin...,Although recent genome-wide studies have prov...,0.116110
8553,8609,The Origin of Species,{It's hard to talk about <I>The Origin of Spec...,0.114067
15767,15968,Analyzing biological network parameters with C...,Summary: The increasing availability of large ...,0.113460
14267,14425,Cavity Optomechanics: Back-Action at the Mesos...,The coupling of optical and mechanical degrees...,0.111159
5326,5360,Using latent semantic analysis to identify sim...,The paper describes the results of applying La...,0.107685
10162,10236,Repression of protein synthesis by miRNAs: how...,MicroRNAs are 21-nucleotide-long regulators o...,0.103802
3876,3899,Analysis of molecular variance inferred from m...,We present here a framework for the study of m...,0.100829
10970,11054,Statistical significance of quantitative PCR.,ABSTRACT: BACKGROUND: PCR has the potential to...,0.098031
9351,9418,MEME: discovering and analyzing DNA and protei...,{MEME} {(Multiple} {EM} for Motif Elicitation)...,0.095402


In [ ]:
C=recommendations['doc.id'].head(10).tolist()

In [ ]:
A=UR['liked_articles'].iloc[user_row_number]

In [ ]:
sorted(C)

In [ ]:
DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].index

In [ ]:
A = list(map(int, A))

In [ ]:
sorted(A)

In [ ]:
set(C) & set(DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].index)

In [ ]:
# k = 200
recommendations.head(N)

In [ ]:
# k = 85
recommendations.head(N)

In [ ]:
# k = 85
recommendations.head(N)

In [ ]:
# k = 85
recommendations.head(N)

In [ ]:
# k = 80
recommendations.head(N)

In [ ]:
# k = 75
recommendations.head(N)

In [ ]:
# k=50
recommendations.head(N)

In [ ]:
# k=100
recommendations.head(N)

In [ ]:
# k=400
recommendations.head(N)

In [ ]:
# k=300
recommendations.head(N)

In [ ]:
# k=200
recommendations.head(N)

In [ ]:
recommendations.head(N)

In [ ]:
recommendations.head(N)

In [48]:
recommendations.head(N)

docId = DF2['doc.id']

docId[len(docId)-1]

del CountVectorizer
del Counter
del IdFreq
del LogisticRegression
del TfidfVectorizer
del tfidf_matrix
del nbrs_tfidf
del tfidf_vectorizer

DF2.drop(['citeulike.id','title','citations'],axis=1,inplace=True)

M = np.zeros(shape=(len(UR),len(docId)))

for i, row in UR.iterrows():
    for j in range(len(row['liked_articles'])-1):
        M[i,int(row['liked_articles'][j])]=1

BARRIER();

#del UR
del docId

user_ratings_mean = np.sum(M, axis = 1)

from scipy.sparse.linalg import svds

U, sigma, Vt = svds(M, k = 500)

sigma = np.diag(sigma)

print(U.shape,Vt.shape)

all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))# + 
                              #user_ratings_mean.reshape(-1, 1))/user_ratings_mean.reshape(-1, 1)

preds_df = pd.DataFrame(all_user_predicted_ratings)

## Recommending for user #:

user_row_number=598

sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

preds_df.head()